In [ ]:
# La última versión de spacy-stanza (>1.0) es compatible solo con spacy >=3.0
# Nota: spacy 3.0 incorpora al pepiline nlp transformers
!pip install -U spacy==3.1 --quiet
!pip install -U spacy-stanza==1.0.0 --quiet

In [ ]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
import re
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
nltk.download("punkt")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import stanza
import spacy_stanza

# Vamos a usar SpaCy-Stanza. Stanza es una librería de NLP de Stanford
# SpaCy armó un wrapper para los pipelines y modelos de Stanza
# https://stanfordnlp.github.io/stanza/

# Descargar el diccionario en español y armar el pipeline de NLP con spacy
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

2022-03-18 18:08:38 INFO: Downloading default packages for language: es (Spanish)...
2022-03-18 18:08:41 INFO: File exists: /root/stanza_resources/es/default.zip.
2022-03-18 18:08:49 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-03-18 18:08:49 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-03-18 18:08:49 INFO: Use device: cpu
2022-03-18 18:08:49 INFO: Loading: tokenize
2022-03-18 18:08:49 INFO: Loading: mwt
2022-03-18 18:08:49 INFO: Loading: pos
2022-03-18 18:08:49 INFO: Loading: lemma
2022-03-18 18:08:49 INFO: Loading: depparse
2022-03-18 18:08:50 INFO: Loading: ner
2022-03-18 18:08:51 INFO: Done loading processors!


## **Datos**

Se obtienen los datos del artículo de wikipedia sobre el deporte "fútbol" en catellano.

In [ ]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/F%C3%BAtbol')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [ ]:
# Demos un vistazo
article_text


'el fútbol o futbol[2]\u200b (del inglés británico football, traducido como balompié)[3]\u200b es un deporte de equipo jugado entre dos conjuntos de once jugadores cada uno mientras que algunos árbitros que se ocupan de que las normas se cumplan correctamente. es ampliamente considerado el deporte más popular del mundo, pues lo practican unas 270\xa0millones de personas.[4]\u200b también se le conoce como futbol asociación o fútbol asociación, nombre derivado de the football association, primera federación oficial del mundo en este deporte y que utilizó ese nombre para distinguirlo de otros deportes que incluyen la palabra "futbol" o "fútbol".[5]\u200b en algunos países de habla inglesa también se le conoce como soccer, abreviatura de association, puesto que el nombre de football en esos países se asocia mayoritariamente a otros deportes con esa denominación (principalmente en estados unidos, donde el nombre football aplica para el fútbol americano, un deporte totalmente distinto).\nel

In [ ]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 69567


In [ ]:
import re
import unicodedata

def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [ ]:
nota = preprocess_clean_text(article_text.lower())

In [ ]:
print("Cantidad de caracteres en la nota:", len(nota))

Cantidad de caracteres en la nota: 66571


In [ ]:
nota

NameError: ignored

In [ ]:
# Elimino algunos caracteres que no deberían estar
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)
text = text.replace('\u200b', '')

In [ ]:
# Demos un vistazo
text

'el fútbol o futbol  (del inglés británico football, traducido como balompié)  es un deporte de equipo jugado entre dos conjuntos de once jugadores cada uno mientras que algunos árbitros que se ocupan de que las normas se cumplan correctamente. es ampliamente considerado el deporte más popular del mundo, pues lo practican unas 270 millones de personas.  también se le conoce como futbol asociación o fútbol asociación, nombre derivado de the football association, primera federación oficial del mundo en este deporte y que utilizó ese nombre para distinguirlo de otros deportes que incluyen la palabra "futbol" o "fútbol".  en algunos países de habla inglesa también se le conoce como soccer, abreviatura de association, puesto que el nombre de football en esos países se asocia mayoritariamente a otros deportes con esa denominación (principalmente en estados unidos, donde el nombre football aplica para el fútbol americano, un deporte totalmente distinto). el terreno de juego es rectangular de 

In [ ]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 69142


In [ ]:
corpus = nltk.sent_tokenize(text)

words = nltk.word_tokenize(text)


In [ ]:
# Demos un vistazo
corpus[:10]

['el fútbol o futbol  (del inglés británico football, traducido como balompié)  es un deporte de equipo jugado entre dos conjuntos de once jugadores cada uno mientras que algunos árbitros que se ocupan de que las normas se cumplan correctamente.',
 'es ampliamente considerado el deporte más popular del mundo, pues lo practican unas 270 millones de personas.',
 'también se le conoce como futbol asociación o fútbol asociación, nombre derivado de the football association, primera federación oficial del mundo en este deporte y que utilizó ese nombre para distinguirlo de otros deportes que incluyen la palabra "futbol" o "fútbol".',
 'en algunos países de habla inglesa también se le conoce como soccer, abreviatura de association, puesto que el nombre de football en esos países se asocia mayoritariamente a otros deportes con esa denominación (principalmente en estados unidos, donde el nombre football aplica para el fútbol americano, un deporte totalmente distinto).',
 'el terreno de juego es 

In [ ]:
# Demos un vistazo
words[:20]

['el',
 'fútbol',
 'o',
 'futbol',
 '(',
 'del',
 'inglés',
 'británico',
 'football',
 ',',
 'traducido',
 'como',
 'balompié',
 ')',
 'es',
 'un',
 'deporte',
 'de',
 'equipo',
 'jugado']

In [ ]:
print("Vocabulario:", len(words))

Vocabulario: 12965


Voy a utilizar SnowballStemmer para trajar en español.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

#def perform_lemmatization(tokens):
#    return [lemmatizer.lemmatize(token) for token in tokens]

def perform_lemmatization(tokens):
    return [spanish_stemmer.stem(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
nltk.download('stopwords')
def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stopwords.words("spanish"))

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud

    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Lo siento, no puedo entenderlo"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

In [ ]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

examples = [["Gol"],["¿Cuántos jugadores participan de un partido?"]]
iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical",
    examples=examples
)

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://43257.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Gol


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  % sorted(inconsistent)


¿Cuántos jugadores participan de un partido?


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['algun', 'com', 'contr', 'cuand', 'desd', 'dond', 'durant', 'eram', 'estab', 'estais', 'estam', 'estan', 'estand', 'estaran', 'estaras', 'esteis', 'estem', 'esten', 'estes', 'estuv', 'fuer', 'fues', 'fuim', 'fuist', 'hab', 'habr', 'habran', 'habras', 'hast', 'hem', 'hub', 'mas', 'mia', 'mias', 'mio', 'mios', 'much', 'nad', 'nosotr', 'nuestr', 'par', 'per', 'poc', 'porqu', 'qui', 'seais', 'seam', 'sent', 'ser', 'seran', 'seras', 'si', 'sient', 'sint', 'sobr', 'som', 'suy', 'tambien', 'tant', 'ten', 'tendr', 'tendran', 'tendras', 'teng', 'tien', 'tod', 'tuv', 'tuy', 'vosotr', 'vuestr'] not in stop_words.
  % sorted(inconsistent)
